reference

- https://dreamgonfly.github.io/blog/gan-explained/
- https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/gan/gan.py
- https://subinium.github.io/VanillaGAN/


In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision import datasets, transforms

In [2]:
# Define data preprocessing pipeline
## "Composes several transforms together."
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, ),
                         std=(0.5, ))
])

In [3]:
# Load MNIST data
mnist = datasets.MNIST(root='data', 
                       download=True, 
                       transform=transform)

In [4]:
# Build dataloader
dataloader = DataLoader(mnist,
                        batch_size=60, 
                        shuffle=True)

In [5]:
# Generator

class Generator(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.Linear(in_features=100, out_features=256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(in_features=256, out_features=512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(in_features=512, out_features=1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(in_features=1024, out_features=28*28),
            # 생성자의 마지막 레이어에서는 출력값을 픽셀값의 범위인 -1과 1 사이로 만들어주기 위해 Tanh를 사용.
            nn.Tanh()
        )
        
    def forward(self, inputs):
        return self.main(inputs).view(-1, 1, 28, 28)

In [6]:
# Discriminator

class Discriminator(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(inplace=True),
            nn.Linear(in_features=1024, out_features=512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(inplace=True),
            nn.Linear(in_features=512, out_features=256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(inplace=True),
            nn.Linear(in_features=256, out_features=1),
            nn.Sigmoid()
        )
        
    def forward(self, inputs):
        inputs = inputs.view(-1, 28*28)
        return self.main(inputs)

In [7]:
G = Generator()
D = Discriminator()

In [8]:
# Binary Cross Entropy loss
criterion = nn.BCELoss()

In [9]:
# Optimizer

G_optimizer = Adam(G.parameters(),
                   lr=2e-4,
                   betas=(0.5, 0.999))

D_optimizer = Adam(D.parameters(),
                   lr=2e-4,
                   betas=(0.5, 0.999))

In [10]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

def square_plot(data, path):
    """Take an array of shape (n, height, width) or (n, height, width , 3)
       and visualize each (height, width) thing in a grid of size approx. sqrt(n) by sqrt(n)"""

    if type(data) == list:
	    data = np.concatenate(data)
    # normalize data for display
    data = (data - data.min()) / (data.max() - data.min())

    # force the number of filters to be square
    n = int(np.ceil(np.sqrt(data.shape[0])))

    padding = (((0, n ** 2 - data.shape[0]) ,
                (0, 1), (0, 1))  # add some space between filters
               + ((0, 0),) * (data.ndim - 3))  # don't pad the last dimension (if there is one)
    data = np.pad(data , padding, mode='constant' , constant_values=1)  # pad with ones (white)

    # tilethe filters into an image
    data = data.reshape((n , n) + data.shape[1:]).transpose((0 , 2 , 1 , 3) + tuple(range(4 , data.ndim + 1)))

    data = data.reshape((n * data.shape[1] , n * data.shape[3]) + data.shape[4:])

    plt.imsave(path, data, cmap='gray')

In [11]:
import os

leave_log = True
if leave_log:
    result_dir = 'GAN_generated_images'
    if not os.path.isdir(result_dir):
        os.mkdir(result_dir)

if leave_log:
    train_hist = {}
    train_hist['D_losses'] = []
    train_hist['G_losses'] = []
    generated_images = []
    
z_fixed = Variable(torch.randn(5 * 5, 100), volatile=True)

/Users/wyseo/.pyenv/versions/3.7.7/envs/tutorial-pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  from ipykernel import kernelapp as app


In [14]:
for epoch in range(100):
    
    for real_data, _ in dataloader:
        batch_size = real_data.size(0)
        
        real_data = Variable(real_data)
        
        ###### Train Discriminator ######
        target_real = Variable(torch.ones(batch_size, 1))
        target_fake = Variable(torch.zeros(batch_size, 1))
        
        # pass on real data
        D_result_from_real = D(real_data)
        # Data not closer to 1, gets higer loss
        D_loss_real = criterion(D_result_from_real, target_real)
        
        # create random vector Z to pass to Generator
        z = Variable(torch.randn((batch_size, 100)))
        
        # create fake data from Generator based on the random vector Z
        fake_data = G(z)
        D_result_from_fake = D(fake_data)
        # Data not closer to 0, gets higer loss
        D_loss_fake = criterion(D_result_from_fake, target_fake)
        
        # Discriminator loss
        D_loss = D_loss_real + D_loss_fake
        
        # Clear gradient
        D.zero_grad()
        # Update gradient
        D_loss.backward()
        # Optimize
        D_optimizer.step()

        if leave_log:
            D_losses.append(D_loss.data[0])
        
        ###### Train Generator ######
        # Create random vector Z to feed Generator
        z = Variable(torch.randn((batch_size, 100)))
        # Generate fake data from Generator
        fake_data = G(z)
        # Pass fake data to discriminator
        D_result_from_fake = D(fake_data)
        # Data not closer to 1, gets higher loss
        G_loss = criterion(D_result_from_fake, target_real)
        
        
        # Clear gradient
        G.zero_grad()
        # Update gradient
        G_loss.backward()
        # Optimize
        G_optimizer.step()
        
        if leave_log:
            G_losses.append(G_loss.data[0])
            
    if leave_log:
        train_hist['D_losses'].append(torch.mean(torch.FloatTensor(D_losses)))
        train_hist['G_losses'].append(torch.mean(torch.FloatTensor(G_losses)))

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [60, 256]], which is output 0 of LeakyReluBackward1, is at version 2; expected version 1 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).